In [2]:
import numpy as np
import pandas as pd
import cv2, os, random
from matplotlib import pyplot as plt

from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import SGDClassifier

from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, confusion_matrix

In [3]:
data_dir = 'Chinese_MINST_Dataset'
csv_data = os.path.join(data_dir, 'chinese_mnist.csv')
imgs_dir = os.path.join(data_dir, 'data/data')
data = pd.read_csv(csv_data)
data.head()

suite_id  sample_id  code  value character
0         1          1    10      9         九
1         1         10    10      9         九
2         1          2    10      9         九
3         1          3    10      9         九
4         1          4    10      9         九

In [4]:
imgs_id = data.iloc[:,:3].values
imgs_label = data['value'].tolist()
imgs_name = []

#get image name
for i in range(len(imgs_id)):
    imgs_name.append(f'input_{imgs_id[i][0]}_{imgs_id[i][1]}_{imgs_id[i][2]}.jpg')

In [5]:

def get_train_test_set(imgs_id, imgs_name, train_size, test_size):
    imgs_lists = []

    # according to different value, divide original data into 15 lists
    for i in range(15):
        imgs_lists.append([])
    for i in range(len(imgs_id)):
        imgs_lists[imgs_id[i][2]-1].append(imgs_name[i])

    # Make sure that random sampling is done in a stratified way that each class has 333 pictures in training set
    train_class_size = train_size // 15
    test_class_size = test_size // 15

    # Read the same number of pictures from each list.
    imgs_train_set, imgs_test_set, train_labels, test_labels = [], [], [], []
    for i in range(15):
        train_sample = random.sample(imgs_lists[i], train_class_size)
        imgs_train_set.extend(train_sample)
        
        # Select the test set from the remaining samples.
        remain_sample = set(imgs_lists[i]) - set(train_sample)
        test_sample = random.sample(list(remain_sample), test_class_size)
        imgs_test_set.extend(test_sample)
    
        train_labels.extend([i+1]*train_class_size)
        test_labels.extend([i+1]*test_class_size)

    # Get train set and test set
    train_set = []
    test_set = []
    
    # read images and reshape
    for i in range(len(imgs_train_set)):
        img_name = os.path.join(imgs_dir, imgs_train_set[i])
        img = cv2.imread(img_name, cv2.IMREAD_GRAYSCALE)
        train_set.append(img.reshape(-1))
    for i in range(len(imgs_test_set)):
        img_name = os.path.join(imgs_dir, imgs_test_set[i])
        img = cv2.imread(img_name, cv2.IMREAD_GRAYSCALE)
        test_set.append(img.reshape(-1))

    return train_set, train_labels, test_set, test_labels

In [6]:
train_set, train_labels, test_set, test_labels = get_train_test_set(imgs_id, imgs_name, 5000, 1000)
larger_train_set, larger_train_labels, larger_test_set, larger_test_labels = get_train_test_set(imgs_id, imgs_name, 10000, 1000)

In [7]:
def display_proformance(labels, predict):
    f1 = f1_score(labels, predict, average='macro')
    accuracy = accuracy_score(labels, predict)
    precision = precision_score(labels, predict, average='macro')
    recall = recall_score(labels, predict, average='macro')
    confusion = confusion_matrix(labels, predict)
    
    print('f1_score: ', f1)
    print('accuracy_score: ', accuracy)
    print('precision: ', precision)
    print('recall_score: ', recall)
    print('confusion matrix: \n', confusion)

# KNN Classifier

In [8]:
# knn1's train set has 5000 images, knn2's train set has 10000 images
knn1 = KNeighborsClassifier(n_neighbors=3)
knn1.fit(train_set, train_labels)

knn2 = KNeighborsClassifier(n_neighbors=3)
knn2.fit(larger_train_set, larger_train_labels)

KNeighborsClassifier(n_neighbors=3)

In [12]:
knn1_pred = knn1.predict(test_set)
knn2_pred = knn2.predict(larger_test_set)

print("KNN classifier with 5000 training set:")
display_proformance(test_labels, knn1_pred)
print("KNN classifier with 10000 training set:")
display_proformance(larger_test_labels, knn2_pred)

KNN classifier with 5000 training set:
f1_score:  0.3582794074461379
accuracy_score:  0.34949494949494947
precision:  0.5648109608448737
recall_score:  0.3494949494949494
confusion matrix: 
 [[36  9  4  0  2  1  3  1  0  1  4  0  0  5  0]
 [ 0 63  2  1  0  0  0  0  0  0  0  0  0  0  0]
 [ 0 28 29  9  0  0  0  0  0  0  0  0  0  0  0]
 [ 0 29 21 16  0  0  0  0  0  0  0  0  0  0  0]
 [ 3 32 16  3  8  1  2  0  0  0  0  0  0  1  0]
 [ 0 15 30 15  0  5  0  0  0  0  0  0  0  1  0]
 [ 0 44  1  0  0  0 17  0  4  0  0  0  0  0  0]
 [ 0 32  5  1  0  0  4 14  1  3  4  0  2  0  0]
 [ 0  8  3  0  0  0  1  0 53  0  1  0  0  0  0]
 [ 0 32  4  5  2  2  7  2  0 10  1  0  0  1  0]
 [ 0 19  1  1  0  0  2  2  0  0 35  0  6  0  0]
 [ 1 19  8  5  4  1  0  3  0  1  6 12  1  5  0]
 [ 0 25  4  3  0  1  2  0  0  0 17  0 14  0  0]
 [ 0 37  4  3  0  0  4  0  0  0  0  0  0 18  0]
 [ 0 21 11  3  4  0  2  2  3  2  2  0  0  0 16]]
KNN classifier with 10000 training set:
f1_score:  0.4470592554617338
accuracy_score:  0

### As the training set grows larger, the performance of the KNN classifier improves.

# Decision Tree classifier

In [10]:
# dt1's train set has 5000 images, dt2's train set has 10000 images
dt1 = DecisionTreeClassifier()
dt1.fit(train_set, train_labels)

dt2 = DecisionTreeClassifier()
dt2.fit(train_set, train_labels)

DecisionTreeClassifier()

In [11]:
dt1_pred = dt1.predict(test_set)
dt2_pred = dt2.predict(larger_test_set)

print("DT classifier with 5000 training set:")
display_proformance(test_labels, dt1_pred)
print("DT classifier with 10000 training set:")
display_proformance(larger_test_labels, dt2_pred)

DT classifier with 5000 training set:
f1_score:  0.2809103426941673
accuracy_score:  0.28585858585858587
precision:  0.2787631984959374
recall_score:  0.28585858585858587
confusion matrix: 
 [[30  0  2  0  4  0  4  3  1  6  2  7  2  3  2]
 [ 0 54  3  2  0  0  0  0  2  0  2  0  1  1  1]
 [ 2 11 16 11  3  1  2  2  5  2  1  2  4  2  2]
 [ 2  2  8 19  0  4  4  6  4  2  3  5  3  1  3]
 [ 4  1  8  2 15  7  2  4  2  3  3  6  0  5  4]
 [ 4  1  0  7  1 14  6  8  2  7  2  3  4  5  2]
 [ 2  1  0  7  2  7 12  4  5  6  6  5  4  3  2]
 [ 2  2  3  5  1  3  2 14  5  3  6  1  1  9  9]
 [ 0  4  9  1  1  2 10  1 22  1  5  2  2  1  5]
 [ 5  3  3  3  3  3  8  5  2  5  5  3  7  6  5]
 [ 0  2  2  2  2  2  4  3  2  2 26  1 10  6  2]
 [ 4  3  3  2  4  5  8  2  1  2  1 13  3  8  7]
 [ 3  1  2  5  0  3  3  1  3  1  8  4 23  5  4]
 [ 3  0  3  2  5  5  5  5  2  7  2 15  1  6  5]
 [ 2  0  4  2  8  7  3  3  3  8  0  4  3  5 14]]
DT classifier with 10000 training set:
f1_score:  0.5073065188855554
accuracy_score:  0.

### As the training set grows larger, the performance of the DT classifier improves.

# SGD Classifier

In [13]:
# sgd1's train set has 5000 images, sgd2's train set has 10000 images
sgd1 = SGDClassifier(max_iter=250)
sgd1.fit(train_set, train_labels)

SGDClassifier(max_iter=250)

In [14]:
sgd2 = SGDClassifier(max_iter=250)
sgd2.fit(larger_train_set, larger_train_labels)

SGDClassifier(max_iter=250)

In [15]:
sgd1_pred = sgd1.predict(test_set)
sgd2_pred = sgd2.predict(larger_test_set)

print("SGD classifier with 5000 training set:")
display_proformance(test_labels, sgd1_pred)
print("SGD classifier with 10000 training set:")
display_proformance(larger_test_labels, sgd2_pred)

SGD classifier with 5000 training set:
f1_score:  0.28703742855392034
accuracy_score:  0.28484848484848485
precision:  0.299406597192673
recall_score:  0.28484848484848485
confusion matrix: 
 [[32  0  0  3  6  1  3  0  0  1  1  9  4  5  1]
 [ 1 21  6  8  5  2  2  4  1  3  4  2  1  4  2]
 [ 0  9 14  8  1  3  2  5  7  2  3  1  4  0  7]
 [ 4  1  8 19  2 13  1  2  0  6  0  3  4  1  2]
 [ 0  8  3  0 14  5  4  4  0  5  3  7  4  3  6]
 [ 2  2  4  5  0 19  3  2  1 10  0  8  3  4  3]
 [ 6  1  0  5  4  5 10  1  3  8  2  4  6  2  9]
 [ 2  0  3  3  5  2  5 19  2  7  3  4  3  1  7]
 [ 2  2  5  1  1  4  3  6 18  5  4  3  0  4  8]
 [ 3  0  2  6  3  4  8  7  2 11  0  5  1  6  8]
 [ 3  1  0  1  2  3  4  4  0  3 21  4 16  2  2]
 [ 1  0  3  3  4  8  0  0  0  6  6 23  8  2  2]
 [ 1  0  3  2  4  0  7  1  1  1 10  4 27  1  4]
 [ 3  1  2  0  8  7  4  0  0  9  2  9  5 12  4]
 [ 1  0  0  1  9  1  8  6  2  9  1  4  1  1 22]]
SGD classifier with 10000 training set:
f1_score:  0.3090754761940804
accuracy_score:  

### As the training set grows larger, SGD classifier gets better proformance.
### Among the three classifiers, the performance of K-Nearest Neighbours(KNN) classifie is the best.